EasyOCR

In [4]:
import easyocr

reader = easyocr.Reader(['es']) 

path_img = reader.readtext('matricula.jpg')

for (bbox, text, prob) in path_img:
    # Coordenadas en orden 
    (top_left, top_right, bottom_right, bottom_left) = bbox
    print(f'\nTexto: {text}\nProbabilidad: {prob:.2f}\nContenedor: {tuple(map(int, top_left)),tuple(map(int, bottom_right))}')




Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



Texto: 072L HPH
Probabilidad: 0.40
Contenedor: ((11, 1), (126, 38))


In [3]:
import cv2
import easyocr
from ultralytics import YOLO

# --- 1. CONFIGURACIÓN INICIAL ---

# Carga tu modelo YOLO (asegúrate de que la ruta sea correcta)
# Usará la CPU por defecto. Si tienes GPU: model = YOLO('ruta/a/tu/modelo.pt', device='cuda')
try:
    model = YOLO(r'runs\detect\matricula_yolo11n\weights\best.pt')
except Exception as e:
    print(f"Error cargando el modelo YOLO: {e}")
    exit()

# Carga el modelo GENERAL de YOLO (detecta personas, coches, etc.)
# 'yolov8n.pt' es pequeño y rápido. Puedes usar 'yolov8s.pt' para más precisión.
try:
    model_general = YOLO('yolo11n.pt')
except Exception as e:
    print(f"Error cargando el modelo YOLO general (yolov8n.pt): {e}")
    exit()
# Carga easyOCR
# 'es' para español, 'en' para inglés. Las matrículas suelen ser bien leídas con 'en'
# por los caracteres alfanuméricos. Puedes probar con ['es', 'en'] o solo ['en'].
try:
    reader = easyocr.Reader(['en'], gpu=True) # Pon gpu=False si no tienes GPU o da problemas
except Exception as e:
    print(f"Error cargando easyOCR: {e}")
    exit()
    
# Define los caracteres permitidos en una matrícula (ajusta según tu región)
# Esto mejora MUCHO la precisión y evita caracteres sin sentido.
ALLOWED_CHARACTERS = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'

classNames = [0,2]

# Abre el archivo de video
video_path = r'C:\Users\Alfredo\Downloads\C0142.MP4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: No se pudo abrir el video en {video_path}")
    exit()



# --- ### AÑADIDO: 1. CONFIGURACIÓN DEL VIDEO DE SALIDA ### ---
# Obtenemos las propiedades del video original
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define el códec y crea el objeto VideoWriter
# 'mp4v' es un códec común para .mp4
output_path = 'video_salida_procesado.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
print(f"Guardando video procesado en: {output_path}")

# --- 2. BUCLE PRINCIPAL DE PROCESAMIENTO ---

while cap.isOpened():
    # Lee un frame del video
    ret, frame = cap.read()
    if not ret:
        print("Fin del video o error al leer el frame.")
        break

    # --- 3. DETECCIÓN CON YOLO ---
    # Ejecuta la predicción de YOLO en el frame
    # results es una lista, tomamos el primer elemento [0]
    try:
        results = model(frame, verbose=False)[0] 
    except Exception as e:
        print(f"Error durante la predicción de YOLO: {e}")
        continue

# --- 4. DETECCIÓN DE PERSONAS Y COCHES (Modelo 2) --- ### AÑADIDO ###
    try:
        results_general = model_general(frame, classes=classNames, verbose=False)[0]
    except Exception as e:
        print(f"Error en predicción general: {e}")
        continue

    # Itera sobre todas las detecciones
    for box in results.boxes.data:
        # Extrae coordenadas, confianza (conf) y clase (cls)
        x1, y1, x2, y2, conf, cls = box
        
        # Filtra por confianza (ajusta este umbral según tus pruebas)
        if conf > 0.5:
            # Convierte coordenadas a enteros
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # --- 4. RECORTAR LA MATRÍCULA (ROI) ---
            # Recorta la región de interés (la matrícula) del frame original
            try:
                matricula_img = frame[y1:y2, x1:x2]
                
                if matricula_img.size == 0:
                    continue # Evita procesar imágenes vacías

                # --- 5. EJECUTAR easyOCR SOBRE EL RECORTE ---
                
                # (Opcional pero recomendado) Pre-procesado simple
                # Convertir a escala de grises puede ayudar a easyOCR
                matricula_gray = cv2.cvtColor(matricula_img, cv2.COLOR_BGR2GRAY)

                # Ejecuta easyOCR
                # detail=0: devuelve solo el texto
                # allowlist: filtra los caracteres
                ocr_result = reader.readtext(
                    matricula_gray, 
                    detail=0,
                    allowlist=ALLOWED_CHARACTERS,
                    paragraph=False # Trata cada línea como texto separado
                )
                
                # Une los resultados (a veces easyOCR divide una matrícula en 2 líneas)
                if ocr_result:
                    texto_matricula = " ".join(ocr_result).upper().replace(" ", "")
                else:
                    texto_matricula = ""

                # --- 6. VISUALIZAR RESULTADOS ---
                
                # Dibuja la caja de YOLO
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Prepara el texto para mostrar (matrícula + confianza)
                label = f"{texto_matricula} (Conf: {conf:.2f})"
                
                # Coloca el texto de la matrícula encima de la caja
                cv2.putText(frame, label, (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                            
            except Exception as e:
                print(f"Error procesando OCR o dibujando: {e}")

    for box in results_general.boxes.data:
        x1, y1, x2, y2, conf, cls = box
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cls = int(cls)

        # Asignar nombre de clase y color
        if cls == 0:
            nombre_clase = 'Persona'
            color = (255, 0, 0) # Azul
        elif cls == 2:
            nombre_clase = 'Coche'
            color = (0, 0, 255) # Rojo
        else:
            continue # Por si acaso

        # Dibujar la caja
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        
        # Poner etiqueta
        label_general = f"{nombre_clase} ({conf:.2f})"
        cv2.putText(frame, label_general, (x1, y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)


    # Muestra el frame procesado
    cv2.imshow('YOLO11 + matriculas + easyOCR', frame)

    out.write(frame)
    # Condición de salida (presiona 'q' para salir)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- 7. LIMPIEZA ---
cap.release()
out.release()
cv2.destroyAllWindows()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Guardando video procesado en: video_salida_procesado.mp4


d:\anaconda\envs\VC_P4_EasyOCR\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Fin del video o error al leer el frame.


In [5]:
import cv2
import easyocr
from ultralytics import YOLO
import csv
import os

# --- 1. CONFIGURACIÓN INICIAL ---

# Modelo 1: Tu modelo de MATRÍCULAS
try:
    # Asegúrate de poner la ruta correcta a tu modelo de matrículas
    model_placas = YOLO(r'runs\detect\matricula_yolo11n\weights\best.pt') 
except Exception as e:
    print(f"Error cargando tu modelo de matrículas: {e}")
    exit()

# Modelo 2: Tu modelo 'yolo11n.pt' (para personas/coches)
try:
    model_general = YOLO('yolo11n.pt') 
except Exception as e:
    print(f"Error cargando el modelo 'yolo11n.pt': {e}")
    exit()

# Carga easyOCR
try:
    reader = easyocr.Reader(['en'], gpu=True) 
except Exception as e:
    print(f"Error cargando easyOCR: {e}")
    exit()
    
ALLOWED_CHARACTERS = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'

# --- ### ¡IMPORTANTE! REVISA ESTO ### ---
# Revisa el .yaml de tu modelo 'yolo11n.pt' para saber los IDs correctos.
# Estoy asumiendo 0='persona', 1='coche'. ¡AJÚSTALO!
CLASES_GENERALES_INTERES = [0, 2] 
ID_PERSONA = 0  # <-- Cambia esto
ID_COCHE = 2    # <-- Cambia esto
# --- ### FIN DEL BLOQUE IMPORTANTE ### ---


# Abre el video
video_path = r'C:\Users\Alfredo\Downloads\C0142.MP4' 
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: No se pudo abrir el video en {video_path}")
    exit()

# --- 2. CONFIGURACIÓN DEL VIDEO DE SALIDA ---
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_path = 'video_salida_tracking.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# --- 3. CONFIGURACIÓN DEL ARCHIVO CSV ---
csv_path = 'resultados_tracking.csv'
csv_header = [
    'fotograma', 
    'track_id', 
    'tipo_objeto', 
    'confianza', 
    'x1', 'y1', 'x2', 'y2', 
    'texto_matricula'
]

try:
    csv_file = open(csv_path, 'w', newline='', encoding='utf-8')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(csv_header)
except IOError as e:
    print(f"Error abriendo el archivo CSV: {e}")
    exit()

print(f"Guardando video en: {output_path}")
print(f"Guardando datos CSV en: {csv_path}")

frame_counter = 0 

# --- 4. BUCLE PRINCIPAL DE PROCESAMIENTO ---
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Fin del video.")
        break
        
    frame_counter += 1

    # --- 5. DETECCIÓN Y TRACKING (PERSONAS Y COCHES - Modelo 'yolo11n.pt') ---
    try:
        results_general = model_general.track(
            frame, 
            classes=CLASES_GENERALES_INTERES, 
            persist=True, 
            verbose=False
        )[0]
    except Exception as e:
        print(f"Error en tracking general: {e}")
        continue

    # --- 6. DETECCIÓN Y TRACKING (MATRÍCULAS - Tu modelo de placas) ---
    try:
        results_placas = model_placas.track(
            frame, 
            persist=True, 
            verbose=False
        )[0]
    except Exception as e:
        print(f"Error en tracking de matrículas: {e}")
        continue

    # --- 7. PROCESAR Y GUARDAR PERSONAS/COCHES ---
    if results_general.boxes.id is not None:
        boxes = results_general.boxes
        for i in range(len(boxes)):
            box = boxes[i]
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            track_id = int(box.id[0]) 

            # --- ### ¡IMPORTANTE! REVISA ESTO ### ---
            # Asegúrate de que estos 'if' coincidan con tus IDs
            if cls == ID_PERSONA:
                nombre_clase = 'persona'
                color = (255, 0, 0) # Azul
            elif cls == ID_COCHE:
                nombre_clase = 'coche'
                color = (0, 0, 255) # Rojo
            else:
                continue
            
            # Dibujar caja y ID de tracking
            label = f"ID: {track_id} {nombre_clase}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # Escribir en el CSV
            csv_row = [frame_counter, track_id, nombre_clase, f"{conf:.2f}", x1, y1, x2, y2, '']
            csv_writer.writerow(csv_row)
            
    # --- 8. PROCESAR Y GUARDAR MATRÍCULAS (CON OCR) ---
    if results_placas.boxes.id is not None:
        boxes_placas = results_placas.boxes
        for i in range(len(boxes_placas)):
            box = boxes_placas[i]
            x1_placa, y1_placa, x2_placa, y2_placa = map(int, box.xyxy[0])
            conf_placa = float(box.conf[0])
            track_id_placa = int(box.id[0])
            nombre_clase_placa = 'matricula' 
            
            texto_matricula = "" 
            try:
                matricula_img = frame[y1_placa:y2_placa, x1_placa:x2_placa]
                if matricula_img.size > 0:
                    matricula_gray = cv2.cvtColor(matricula_img, cv2.COLOR_BGR2GRAY)
                    ocr_result = reader.readtext(
                        matricula_gray, 
                        detail=0,
                        allowlist=ALLOWED_CHARACTERS,
                        paragraph=False 
                    )
                    if ocr_result:
                        texto_matricula = "".join(ocr_result).upper()
            except Exception as e:
                print(f"Error en OCR: {e}") 

            # Dibujar caja y texto de matrícula
            color_placa = (0, 255, 0) # Verde
            label_placa = f"ID: {track_id_placa} [{texto_matricula}]"
            cv2.rectangle(frame, (x1_placa, y1_placa), (x2_placa, y2_placa), color_placa, 2)
            cv2.putText(frame, label_placa, (x1_placa, y1_placa - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color_placa, 2)

            # Escribir en el CSV
            csv_row_placa = [frame_counter, track_id_placa, nombre_clase_placa, f"{conf_placa:.2f}", x1_placa, y1_placa, x2_placa, y2_placa, texto_matricula]
            csv_writer.writerow(csv_row_placa)


    # --- 9. MOSTRAR FRAME Y GUARDAR ---
    out.write(frame)
    cv2.imshow('Deteccion con Tracking (YOLO + easyOCR)', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- 10. LIMPIEZA ---
print("Limpiando y guardando archivos...")
cap.release()
out.release() 
csv_file.close() 
cv2.destroyAllWindows()
print("¡Proceso completado!")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Guardando video en: video_salida_tracking.mp4
Guardando datos CSV en: resultados_tracking.csv
Fin del video.
Limpiando y guardando archivos...
¡Proceso completado!
